# HEART DISEASE PREDICTION

### Importing Libraries

In [36]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz  # to export graph of decision tree to pdf
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import accuracy_score, classification_report
import warnings
warnings.filterwarnings("ignore")

# Data Collection and Processing 

**About Dataset**  
**Context**  
This data set dates from 1988 and consists of four databases: Cleveland, Hungary, Switzerland, and Long Beach V. It contains 76 attributes, including the predicted attribute, but all published experiments refer to using a subset of 14 of them. The "target" field refers to the presence of heart disease in the patient. It is integer valued 0 = no disease and 1 = disease.  

**Content**  
Attribute Information:  

- age  
- sex  
- chest pain type (4 values)  
- resting blood pressure  
- serum cholestoral in mg/dl  
- fasting blood sugar > 120 mg/dl  
- resting electrocardiographic results (values 0,1,2)  
- maximum heart rate achieved  
- exercise induced angina  
- oldpeak = ST depression induced by exercise relative to rest  
- the slope of the peak exercise ST segment  
- number of major vessels (0-3) colored by flourosopy  
- thal: 0 = normal; 1 = fixed defect; 2 = reversable defect  
- The names and social security numbers of the patients were recently removed from the database, replaced with dummy values.  

In [2]:
# loading the csv data to a Pandas DataFrame
heart_data = pd.read_csv('heart.csv')

In [3]:
# print first 5 rows of the dataset
heart_data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0


In [4]:
# print last 5 rows of the dataset
heart_data.tail()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
1020,59,1,1,140,221,0,1,164,1,0.0,2,0,2,1
1021,60,1,0,125,258,0,0,141,1,2.8,1,1,3,0
1022,47,1,0,110,275,0,0,118,1,1.0,1,1,2,0
1023,50,0,0,110,254,0,0,159,0,0.0,2,0,2,1
1024,54,1,0,120,188,0,1,113,0,1.4,1,1,3,0


In [5]:
# number of rows and columns in the dataset
heart_data.shape

(1025, 14)

In [6]:
# getting some info about the data
heart_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1025 entries, 0 to 1024
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1025 non-null   int64  
 1   sex       1025 non-null   int64  
 2   cp        1025 non-null   int64  
 3   trestbps  1025 non-null   int64  
 4   chol      1025 non-null   int64  
 5   fbs       1025 non-null   int64  
 6   restecg   1025 non-null   int64  
 7   thalach   1025 non-null   int64  
 8   exang     1025 non-null   int64  
 9   oldpeak   1025 non-null   float64
 10  slope     1025 non-null   int64  
 11  ca        1025 non-null   int64  
 12  thal      1025 non-null   int64  
 13  target    1025 non-null   int64  
dtypes: float64(1), int64(13)
memory usage: 112.2 KB


In [7]:
# checking for missing values
heart_data.isnull().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

In [8]:
# statistical measures about the data
heart_data.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,1025.000000,1025.000000,1025.000000,1025.000000,1025.00000,1025.000000,1025.000000,1025.000000,1025.000000,1025.000000,1025.000000,1025.000000,1025.000000,1025.000000
mean,54.434146,0.695610,0.942439,131.611707,246.00000,0.149268,0.529756,149.114146,0.336585,1.071512,1.385366,0.754146,2.323902,0.513171
std,9.072290,0.460373,1.029641,17.516718,51.59251,0.356527,0.527878,23.005724,0.472772,1.175053,0.617755,1.030798,0.620660,0.500070
min,29.000000,0.000000,0.000000,94.000000,126.00000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,48.000000,0.000000,0.000000,120.000000,211.00000,0.000000,0.000000,132.000000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,56.000000,1.000000,1.000000,130.000000,240.00000,0.000000,1.000000,152.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,275.00000,0.000000,1.000000,166.000000,1.000000,1.800000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.00000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


# Data Scaling

In [9]:
# checking the distribution of Target Variable
heart_data['target'].value_counts()

1    526
0    499
Name: target, dtype: int64

1 --> Defective Heart

0 --> Healthy Heart

Splitting the Features and Target

In [10]:
X = heart_data.drop(columns='target', axis=1)
Y = heart_data['target']

In [11]:
# scaler = MinMaxScaler()
# X = pd.DataFrame(scaler.fit_transform(X) , columns = X.columns)
# X

In [12]:
Y

0       0
1       0
2       0
3       0
4       0
       ..
1020    1
1021    0
1022    0
1023    1
1024    0
Name: target, Length: 1025, dtype: int64

In [12]:
#Splitting Data into Training data & Testing Data

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

In [13]:
print(X.shape, X_train.shape, X_test.shape)

(1025, 13) (820, 13) (205, 13)


# Model Training & Evaluation

### Logistic Regression

In [14]:
# Scaling data for Logistic Regression
scaler = MinMaxScaler()
X = pd.DataFrame(scaler.fit_transform(X) , columns = X.columns)
X_trainLR, X_testLR, Y_trainLR, Y_testLR = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

In [15]:
logistic_reg = LogisticRegression(random_state = 0)
logistic_reg.fit(X_trainLR, Y_trainLR) # Training

LogisticRegression(random_state=0)

In [17]:
# accuracy on training data
X_train_prediction1 = logistic_reg.predict(X_trainLR)
training_data_accuracy = accuracy_score(X_train_prediction1, Y_trainLR)

print('Accuracy on Training data : ', training_data_accuracy)

Accuracy on Training data :  0.8621951219512195


In [18]:
# accuracy on test data
X_test_prediction1 = logistic_reg.predict(X_testLR)
test_data_accuracy = accuracy_score(X_test_prediction1, Y_testLR)
print('Accuracy on Test data : ', test_data_accuracy)

Accuracy on Test data :  0.8195121951219512


In [19]:
# Confusion Matrix
print(classification_report(Y_testLR , X_test_prediction1))

              precision    recall  f1-score   support

           0       0.85      0.76      0.80       100
           1       0.79      0.88      0.83       105

    accuracy                           0.82       205
   macro avg       0.82      0.82      0.82       205
weighted avg       0.82      0.82      0.82       205



### Decision Tree Classifier

In [16]:
dec_tree_clf = DecisionTreeClassifier(random_state=0, max_depth = 5 , min_samples_leaf = 1 , min_samples_split = 5) 
dec_tree_clf.fit(X_train, Y_train)  # fits the data

DecisionTreeClassifier(max_depth=5, min_samples_split=5, random_state=0)

In [17]:
# accuracy on training data
X_train_prediction2 = dec_tree_clf.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction2, Y_train)
print('Accuracy on Training data : ', training_data_accuracy)

Accuracy on Training data :  0.9365853658536586


In [18]:
# accuracy on test data
X_test_prediction2 = dec_tree_clf.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction2, Y_test)
print('Accuracy on Test data : ', test_data_accuracy)

Accuracy on Test data :  0.8829268292682927


In [19]:
# Confusion Matrix
print(classification_report(Y_test , X_test_prediction2))

              precision    recall  f1-score   support

           0       0.87      0.90      0.88       100
           1       0.90      0.87      0.88       105

    accuracy                           0.88       205
   macro avg       0.88      0.88      0.88       205
weighted avg       0.88      0.88      0.88       205



### Random Forest Classifier

In [25]:
random_forest_clf = RandomForestClassifier(max_depth = 6, random_state=0)  # creating object or instance
random_forest_clf.fit(X_train, Y_train)  # fits the data

RandomForestClassifier(max_depth=6, random_state=0)

In [26]:
# accuracy on training data
X_train_prediction3 = random_forest_clf.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction3, Y_train)
print('Accuracy on Training data : ', training_data_accuracy)

Accuracy on Training data :  0.973170731707317


In [27]:
# accuracy on test data
X_test_prediction3 = random_forest_clf.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction3, Y_test)
print('Accuracy on Test data : ', test_data_accuracy)

Accuracy on Test data :  0.9707317073170731


In [28]:
# Confusion Matrix
print(classification_report(Y_test , X_test_prediction3))

              precision    recall  f1-score   support

           0       0.98      0.96      0.97       100
           1       0.96      0.98      0.97       105

    accuracy                           0.97       205
   macro avg       0.97      0.97      0.97       205
weighted avg       0.97      0.97      0.97       205



### Grid - Search CV - Hyperparameter Tuning

In [39]:
param_grid = { 
    'n_estimators': [200, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [3,4,5,6],
}
CV_rfc = GridSearchCV(estimator=random_forest_clf, param_grid = param_grid)
CV_rfc.fit(X_train, Y_train)
CV_rfc.best_params_

{'max_depth': 6, 'max_features': 'sqrt', 'n_estimators': 200}

In [41]:
random_forest_clf = RandomForestClassifier(max_depth = 6, random_state=0, max_features='sqrt', n_estimators=200)
random_forest_clf.fit(X_train, Y_train)
X_test_prediction3 = random_forest_clf.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction3, Y_test)
print('Accuracy on Test data : ', test_data_accuracy)

Accuracy on Test data :  0.9853658536585366


# Defining our Predictive System

In [42]:
input_data = (67,0,0,106,223,0,1,142,0,0.3,2,2,2)

# change the input data to a numpy array
input_data_as_numpy_array= np.asarray(input_data)

# reshape the numpy array as we are predicting for only on instance
input_data_reshaped = input_data_as_numpy_array.reshape(1,-1)

prediction = random_forest_clf.predict(input_data_reshaped)
print(prediction)

if (prediction[0]== 0):
    print('The Person does not have a Heart Disease')
else:
    print('The Person has Heart Disease')

[1]
The Person has Heart Disease


# Deploying Model

In [47]:
import pickle

In [48]:
filename = 'heart_disease_model.sav'
pickle.dump(random_forest_clf, open(filename, 'wb')) #saving file in directory with extension -> .pkl
                                     # wb - writing file in Binary Mode so that the data is returned in a bytes object.

In [49]:
# loading the saved model
loaded_model = pickle.load(open('heart_disease_model.sav', 'rb'))  #unpickle file using load() function
                                # deserialize the file, we need to use the “rb” mode which stands for read binary.

In [50]:
for column in X.columns:
    print(column)

age
sex
cp
trestbps
chol
fbs
restecg
thalach
exang
oldpeak
slope
ca
thal
